# Antimicrobial Treatment Options and Stewardship Recommendations

**Research Question 3:** What are the effective treatment options for common infections based on current resistance patterns?

This analysis provides evidence-based treatment recommendations for clinicians, focusing on:
- First-line antibiotic selection based on high sensitivity rates
- Reserve antibiotics for multi-drug resistant (MDR) organisms
- Infection-specific treatment algorithms
- Antimicrobial stewardship implications

**Clinical Perspective:** Treatment recommendations are derived from local susceptibility data to optimize empiric therapy while preserving critical antimicrobials.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
%matplotlib inline
sns.set_style('whitegrid')
sns.set_palette('Set2')

## 1. Load and Prepare Data

In [ ]:
# Load cleaned AMR data
df = pd.read_csv('../data/processed/amr_data_2025_cleaned.csv')

# Load resistance rates summary
try:
    resistance_summary = pd.read_csv('../data/processed/resistance_rates_summary.csv')
    print("Loaded pre-computed resistance summary")
except FileNotFoundError:
    print("Resistance summary not found. Will compute from raw data.")
    resistance_summary = None

# Identify antibiotic columns
antibiotic_cols = [col for col in df.columns if ' - ' in col or col.startswith('NET_') or col.startswith('MET_')]

# Categorize results
def categorize_result(result):
    if pd.isna(result):
        return np.nan
    result_str = str(result).upper()
    if 'R' in result_str:
        return 'Resistant'
    elif 'S' in result_str:
        return 'Sensitive'
    elif 'I' in result_str:
        return 'Intermediate'
    return np.nan

for col in antibiotic_cols:
    df[col + '_Cat'] = df[col].apply(categorize_result)

print(f"Dataset: {len(df)} isolates")
print(f"Antibiotics: {len(antibiotic_cols)}")
print(f"Sample types: {df['Sample Type'].value_counts().to_dict()}")

## 2. Calculate Sensitivity Rates for Treatment Selection

**Clinical Threshold:** Antibiotics with ≥80% sensitivity are considered appropriate for empiric first-line therapy.

In [ ]:
# Calculate sensitivity rates if not already loaded
if resistance_summary is None:
    resistance_data = []
    
    for col in antibiotic_cols:
        cat_col = col + '_Cat'
        
        total = df[cat_col].notna().sum()
        if total < 30:
            continue
        
        resistant = (df[cat_col] == 'Resistant').sum()
        sensitive = (df[cat_col] == 'Sensitive').sum()
        intermediate = (df[cat_col] == 'Intermediate').sum()
        
        resistance_rate = (resistant / total) * 100
        sensitivity_rate = (sensitive / total) * 100
        
        ab_name = col.split(' - ')[1] if ' - ' in col else col.replace('_', ' ')
        
        resistance_data.append({
            'Antibiotic': ab_name,
            'Total_Tests': total,
            'Resistant': resistant,
            'Sensitive': sensitive,
            'Intermediate': intermediate,
            'Resistance_Rate': resistance_rate,
            'Sensitivity_Rate': sensitivity_rate
        })
    
    resistance_summary = pd.DataFrame(resistance_data)

# Sort by sensitivity rate
resistance_summary = resistance_summary.sort_values('Sensitivity_Rate', ascending=False)

print("\n=== ANTIBIOTIC SENSITIVITY OVERVIEW ===")
print(f"Total antibiotics analyzed: {len(resistance_summary)}")
print(f"\nFirst-line options (≥80% sensitivity): {(resistance_summary['Sensitivity_Rate'] >= 80).sum()}")
print(f"Moderate options (60-79% sensitivity): {((resistance_summary['Sensitivity_Rate'] >= 60) & (resistance_summary['Sensitivity_Rate'] < 80)).sum()}")
print(f"Limited utility (<60% sensitivity): {(resistance_summary['Sensitivity_Rate'] < 60).sum()}")

## 3. First-Line Antibiotic Recommendations (>80% Sensitivity)

These antibiotics demonstrate high sensitivity rates and are suitable for empiric therapy.

In [ ]:
# Identify first-line antibiotics
first_line = resistance_summary[resistance_summary['Sensitivity_Rate'] >= 80].copy()
first_line['Clinical_Class'] = first_line['Antibiotic'].apply(lambda x: 
    'Carbapenem' if any(c in x for c in ['Imipenem', 'Meropenem', 'Ertapenem', 'Doripenem']) else
    'Aminoglycoside' if any(c in x for c in ['Gentamicin', 'Amikacin', 'Tobramycin']) else
    'Glycopeptide' if any(c in x for c in ['Vancomycin', 'Teicoplanin']) else
    'Fluoroquinolone' if any(c in x for c in ['Ciprofloxacin', 'Levofloxacin', 'Moxifloxacin']) else
    'Cephalosporin' if 'Cef' in x else
    'Beta-lactam/BLI' if any(c in x for c in ['Clavulanic', 'Tazobactam', 'Sulbactam']) else
    'Other'
)

print("\n=== FIRST-LINE ANTIBIOTICS (≥80% Sensitivity) ===")
print("\nRanked by sensitivity rate:\n")
print(first_line[['Antibiotic', 'Clinical_Class', 'Sensitivity_Rate', 'Resistance_Rate', 'Total_Tests']].to_string(index=False))

# Visualize first-line options
if len(first_line) > 0:
    fig, ax = plt.subplots(figsize=(12, max(6, len(first_line) * 0.4)))
    
    colors = {'Carbapenem': '#e74c3c', 'Aminoglycoside': '#3498db', 
              'Glycopeptide': '#9b59b6', 'Fluoroquinolone': '#f39c12',
              'Cephalosporin': '#1abc9c', 'Beta-lactam/BLI': '#2ecc71',
              'Other': '#95a5a6'}
    
    bar_colors = [colors.get(c, '#95a5a6') for c in first_line['Clinical_Class']]
    
    bars = ax.barh(range(len(first_line)), first_line['Sensitivity_Rate'], 
                   color=bar_colors, alpha=0.8, edgecolor='black', linewidth=0.5)
    ax.set_yticks(range(len(first_line)))
    ax.set_yticklabels(first_line['Antibiotic'])
    ax.set_xlabel('Sensitivity Rate (%)', fontsize=12, fontweight='bold')
    ax.set_title('First-Line Treatment Options (≥80% Sensitivity)', 
                 fontsize=14, fontweight='bold', pad=20)
    ax.invert_yaxis()
    ax.set_xlim(0, 105)
    
    # Add value labels
    for i, (rate, n, cls) in enumerate(zip(first_line['Sensitivity_Rate'], 
                                            first_line['Total_Tests'],
                                            first_line['Clinical_Class'])):
        ax.text(rate + 1, i, f'{rate:.1f}% (n={n})', 
                va='center', fontsize=9, fontweight='bold')
    
    # Add legend
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor=color, label=cls, alpha=0.8) 
                      for cls, color in colors.items() 
                      if cls in first_line['Clinical_Class'].values]
    ax.legend(handles=legend_elements, loc='lower right', title='Antibiotic Class')
    
    ax.axvline(80, color='green', linestyle='--', alpha=0.5, linewidth=2, label='80% threshold')
    
    plt.tight_layout()
    plt.savefig('../reports/figures/first_line_antibiotics.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("\n⚠️ WARNING: No antibiotics meet first-line criteria (≥80% sensitivity)")
    print("This indicates severe resistance and requires immediate antimicrobial stewardship intervention.")

## 4. Reserve Antibiotics for MDR Cases

**Reserve antibiotics** should be preserved for confirmed MDR infections to minimize selection pressure and preserve their efficacy.

In [ ]:
# Define reserve antibiotics (critical antimicrobials per WHO AWaRe classification)
reserve_antibiotics = [
    'Meropenem', 'Imipenem', 'Ertapenem', 'Doripenem',  # Carbapenems
    'Colistin', 'Polymyxin B',  # Polymyxins
    'Tigecycline',  # Glycylcyclines
    'Linezolid', 'Tedizolid',  # Oxazolidinones
    'Daptomycin',  # Lipopeptides
    'Ceftazidime/Avibactam', 'Ceftolozane/Tazobactam',  # Advanced beta-lactam combinations
]

# Filter reserve antibiotics from our dataset
reserve_in_data = resistance_summary[resistance_summary['Antibiotic'].apply(
    lambda x: any(ra in x for ra in reserve_antibiotics)
)].copy()

print("\n=== RESERVE ANTIBIOTICS (Last-Line Agents) ===")
print("\nThese should be reserved for confirmed MDR infections or when first-line options fail.\n")

if len(reserve_in_data) > 0:
    print(reserve_in_data[['Antibiotic', 'Sensitivity_Rate', 'Resistance_Rate', 'Total_Tests']].to_string(index=False))
    
    # Clinical interpretation
    print("\n--- Clinical Guidance ---")
    for _, row in reserve_in_data.iterrows():
        ab = row['Antibiotic']
        sens = row['Sensitivity_Rate']
        if sens >= 90:
            status = "EXCELLENT efficacy - reserve for documented MDR"
        elif sens >= 80:
            status = "GOOD efficacy - use judiciously for MDR"
        elif sens >= 70:
            status = "MODERATE efficacy - consider alternatives when possible"
        else:
            status = "⚠️ CONCERNING - resistance emerging, urgent stewardship needed"
        print(f"{ab}: {status}")
    
    # Visualize reserve antibiotic performance
    fig, ax = plt.subplots(figsize=(10, max(4, len(reserve_in_data) * 0.5)))
    
    x = np.arange(len(reserve_in_data))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, reserve_in_data['Sensitivity_Rate'], width, 
                   label='Sensitive', color='#2ecc71', alpha=0.8)
    bars2 = ax.bar(x + width/2, reserve_in_data['Resistance_Rate'], width, 
                   label='Resistant', color='#e74c3c', alpha=0.8)
    
    ax.set_ylabel('Rate (%)', fontsize=12, fontweight='bold')
    ax.set_title('Reserve Antibiotic Susceptibility Profile', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(reserve_in_data['Antibiotic'], rotation=45, ha='right')
    ax.legend()
    ax.set_ylim(0, 110)
    
    # Add value labels
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.1f}%', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.savefig('../reports/figures/reserve_antibiotics.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("No reserve antibiotics found in dataset.")
    print("\nNote: If carbapenems or other critical agents were tested, ensure data labeling is consistent.")

## 5. Infection-Specific Treatment Recommendations

Treatment recommendations stratified by infection site and organism profile.

In [ ]:
# Analyze by sample type (infection site)
sample_types = df['Sample Type'].unique()

print("\n=== INFECTION-SPECIFIC ANALYSIS ===")

infection_recommendations = {}

for sample_type in sample_types:
    if pd.isna(sample_type):
        continue
    
    sample_df = df[df['Sample Type'] == sample_type]
    n_samples = len(sample_df)
    
    if n_samples < 10:  # Skip if too few samples
        continue
    
    print(f"\n{'='*70}")
    print(f"INFECTION SITE: {sample_type.upper()}")
    print(f"{'='*70}")
    print(f"Total isolates: {n_samples}\n")
    
    # Common organisms for this infection type
    organisms = sample_df['Organism Identified'].value_counts().head(5)
    print("Common pathogens:")
    for org, count in organisms.items():
        pct = count/n_samples*100
        print(f"  • {org}: {count} ({pct:.1f}%)")
    
    # Calculate sensitivity rates for this infection type
    infection_sensitivity = []
    
    for col in antibiotic_cols:
        cat_col = col + '_Cat'
        total = sample_df[cat_col].notna().sum()
        
        if total < 5:  # Need minimum tests
            continue
        
        sensitive = (sample_df[cat_col] == 'Sensitive').sum()
        resistant = (sample_df[cat_col] == 'Resistant').sum()
        sensitivity_rate = (sensitive / total) * 100
        
        ab_name = col.split(' - ')[1] if ' - ' in col else col.replace('_', ' ')
        
        infection_sensitivity.append({
            'Antibiotic': ab_name,
            'Sensitivity_Rate': sensitivity_rate,
            'Total_Tests': total
        })
    
    infection_sens_df = pd.DataFrame(infection_sensitivity).sort_values(
        'Sensitivity_Rate', ascending=False
    )
    
    # Recommend top antibiotics
    recommended = infection_sens_df[infection_sens_df['Sensitivity_Rate'] >= 80]
    
    print("\nRECOMMENDED EMPIRIC THERAPY (≥80% sensitivity):")
    if len(recommended) > 0:
        for idx, row in recommended.head(5).iterrows():
            print(f"  ✓ {row['Antibiotic']}: {row['Sensitivity_Rate']:.1f}% (n={row['Total_Tests']})")
        
        infection_recommendations[sample_type] = recommended['Antibiotic'].head(5).tolist()
    else:
        print("  ⚠️ No antibiotics meet 80% sensitivity threshold")
        print("  BEST AVAILABLE OPTIONS (highest sensitivity):")
        for idx, row in infection_sens_df.head(3).iterrows():
            print(f"    • {row['Antibiotic']}: {row['Sensitivity_Rate']:.1f}% (n={row['Total_Tests']})")
        
        infection_recommendations[sample_type] = infection_sens_df['Antibiotic'].head(3).tolist()
    
    # Antibiotics to avoid
    avoid = infection_sens_df[infection_sens_df['Sensitivity_Rate'] < 50]
    if len(avoid) > 0:
        print("\nAVOID (high resistance >50%):")
        for idx, row in avoid.head(5).iterrows():
            print(f"  ✗ {row['Antibiotic']}: {row['Sensitivity_Rate']:.1f}% sensitive")

### 5.1 Urinary Tract Infection (UTI) Specific Recommendations

In [ ]:
# Detailed UTI analysis (most common infection)
uti_samples = df[df['Sample Type'].str.contains('Urine', case=False, na=False)]

if len(uti_samples) >= 20:
    print("\n" + "="*70)
    print("DETAILED UTI TREATMENT ALGORITHM")
    print("="*70)
    
    # Stratify by organism
    top_uti_organisms = uti_samples['Organism Identified'].value_counts().head(3)
    
    for organism in top_uti_organisms.index:
        org_df = uti_samples[uti_samples['Organism Identified'] == organism]
        
        print(f"\n--- {organism} UTI (n={len(org_df)}) ---")
        
        # Calculate sensitivity for key UTI antibiotics
        uti_abs = ['Nitrofurantoin', 'Trimethoprim/Sulfamethoxazole', 'Ciprofloxacin', 
                   'Levofloxacin', 'Ceftriaxone', 'Amoxicillin/Clavulanic']
        
        org_recommendations = []
        
        for ab in uti_abs:
            matching_cols = [col for col in antibiotic_cols if ab in col]
            if matching_cols:
                col = matching_cols[0]
                cat_col = col + '_Cat'
                total = org_df[cat_col].notna().sum()
                
                if total >= 3:
                    sensitive = (org_df[cat_col] == 'Sensitive').sum()
                    sens_rate = (sensitive / total) * 100
                    org_recommendations.append((ab, sens_rate, total))
        
        org_recommendations.sort(key=lambda x: x[1], reverse=True)
        
        print("\nRecommended antibiotics (by sensitivity):")
        for ab, sens, n in org_recommendations[:5]:
            if sens >= 80:
                print(f"  1st line: {ab} - {sens:.1f}% (n={n})")
            elif sens >= 60:
                print(f"  2nd line: {ab} - {sens:.1f}% (n={n})")
            else:
                print(f"  Avoid: {ab} - {sens:.1f}% (n={n})")

### 5.2 Organism-Specific Treatment Recommendations

In [ ]:
# Analyze top 5 organisms across all infection types
top_organisms = df['Organism Identified'].value_counts().head(5)

print("\n" + "="*70)
print("ORGANISM-SPECIFIC TREATMENT RECOMMENDATIONS")
print("="*70)

organism_treatment_map = {}

for organism in top_organisms.index:
    org_df = df[df['Organism Identified'] == organism]
    n = len(org_df)
    
    print(f"\n{'─'*70}")
    print(f"{organism} (n={n})")
    print(f"{'─'*70}")
    
    # Calculate sensitivity for this organism
    org_sensitivity = []
    
    for col in antibiotic_cols:
        cat_col = col + '_Cat'
        total = org_df[cat_col].notna().sum()
        
        if total < 5:
            continue
        
        sensitive = (org_df[cat_col] == 'Sensitive').sum()
        resistant = (org_df[cat_col] == 'Resistant').sum()
        sensitivity_rate = (sensitive / total) * 100
        resistance_rate = (resistant / total) * 100
        
        ab_name = col.split(' - ')[1] if ' - ' in col else col.replace('_', ' ')
        
        org_sensitivity.append({
            'Antibiotic': ab_name,
            'Sensitivity_Rate': sensitivity_rate,
            'Resistance_Rate': resistance_rate,
            'Total_Tests': total
        })
    
    org_sens_df = pd.DataFrame(org_sensitivity).sort_values('Sensitivity_Rate', ascending=False)
    
    # Categorize antibiotics
    first_choice = org_sens_df[org_sens_df['Sensitivity_Rate'] >= 85]
    second_choice = org_sens_df[(org_sens_df['Sensitivity_Rate'] >= 70) & 
                                 (org_sens_df['Sensitivity_Rate'] < 85)]
    avoid = org_sens_df[org_sens_df['Resistance_Rate'] >= 50]
    
    print("\nFIRST-LINE (≥85% sensitivity):")
    if len(first_choice) > 0:
        for idx, row in first_choice.head(5).iterrows():
            print(f"  • {row['Antibiotic']}: {row['Sensitivity_Rate']:.1f}%")
        organism_treatment_map[organism] = {'first_line': first_choice['Antibiotic'].tolist()}
    else:
        print("  None available - consider susceptibility-guided therapy")
        organism_treatment_map[organism] = {'first_line': []}
    
    print("\nSECOND-LINE (70-84% sensitivity):")
    if len(second_choice) > 0:
        for idx, row in second_choice.head(3).iterrows():
            print(f"  • {row['Antibiotic']}: {row['Sensitivity_Rate']:.1f}%")
        organism_treatment_map[organism]['second_line'] = second_choice['Antibiotic'].tolist()
    else:
        print("  None available")
        organism_treatment_map[organism]['second_line'] = []
    
    if len(avoid) > 0:
        print("\nAVOID (≥50% resistance):")
        for idx, row in avoid.head(5).iterrows():
            print(f"  ✗ {row['Antibiotic']}: {row['Resistance_Rate']:.1f}% resistant")

## 6. Treatment Decision Algorithm

Clinical decision framework for antibiotic selection based on local resistance patterns.

In [ ]:
# Create treatment algorithm visualization
print("\n" + "="*70)
print("EMPIRIC ANTIBIOTIC SELECTION ALGORITHM")
print("="*70)

algorithm_text = """
┌─────────────────────────────────────────────────────────────┐
│           PATIENT WITH SUSPECTED BACTERIAL INFECTION        │
└──────────────────────┬──────────────────────────────────────┘
                       ↓
              ┌────────────────┐
              │ Identify Site  │
              │  of Infection  │
              └────────┬───────┘
                       ↓
         ┌─────────────┴──────────────┐
         ↓                            ↓
    ┌─────────┐                 ┌──────────┐
    │   UTI   │                 │  Wound/  │
    │         │                 │  Other   │
    └────┬────┘                 └─────┬────┘
         ↓                            ↓
┌────────────────────┐       ┌────────────────────┐
│ Risk Stratification│       │ Risk Stratification│
└────────┬───────────┘       └────────┬───────────┘
         ↓                            ↓
    ┌─────────┐                  ┌─────────┐
    │ Low Risk│                  │High Risk│
    │(No MDR  │                  │(MDR risk│
    │ factors)│                  │ factors)│
    └────┬────┘                  └────┬────┘
         ↓                            ↓
┌──────────────────┐         ┌──────────────────┐
│  FIRST-LINE      │         │  BROAD-SPECTRUM  │
│  ANTIBIOTICS     │         │  or RESERVE      │
│  (≥80% sensitive)│         │  ANTIBIOTICS     │
└──────────────────┘         └──────────────────┘
         ↓                            ↓
┌──────────────────────────────────────────────┐
│        OBTAIN CULTURES & SENSITIVITY         │
└──────────────────┬───────────────────────────┘
                   ↓
┌──────────────────────────────────────────────┐
│    DE-ESCALATE based on culture results      │
│    Narrow spectrum when possible             │
└──────────────────────────────────────────────┘
"""

print(algorithm_text)

# MDR Risk Factors
print("\n" + "─"*70)
print("MDR RISK FACTORS:")
print("─"*70)
print("""
Consider high risk for MDR if ANY of the following:
  • Recent hospitalization (within 90 days)
  • Recent antibiotic use (within 90 days)
  • Healthcare-associated infection
  • Immunocompromised state
  • Prior MDR infection
  • ICU admission
  • Indwelling devices (catheters, central lines)
  • Long-term care facility resident
""")

# Decision points
print("\n" + "─"*70)
print("KEY DECISION POINTS:")
print("─"*70)
print("""
1. EMPIRIC THERAPY SELECTION:
   ✓ Use antibiotics with ≥80% local sensitivity
   ✓ Consider infection site and likely pathogens
   ✓ Assess patient risk factors for resistance

2. CULTURE-GUIDED MODIFICATION:
   ✓ Obtain cultures BEFORE starting antibiotics when possible
   ✓ Review susceptibility results within 48-72 hours
   ✓ De-escalate to narrower spectrum when indicated

3. STEWARDSHIP PRINCIPLES:
   ✓ Avoid empiric use of reserve antibiotics
   ✓ Shortest effective duration
   ✓ IV to PO conversion when clinically appropriate
   ✓ Dose optimization based on pharmacokinetics
""")

## 7. Clinical Treatment Guidelines Summary Table

In [ ]:
# Create comprehensive treatment guideline table
print("\n" + "="*70)
print("CLINICAL TREATMENT GUIDELINES SUMMARY")
print("="*70)

# Build treatment guideline based on available data
treatment_guidelines = []

# UTI Guidelines
if 'Urine' in infection_recommendations:
    treatment_guidelines.append({
        'Infection': 'Uncomplicated UTI',
        'First_Line': ', '.join(infection_recommendations['Urine'][:2]) if len(infection_recommendations['Urine']) >= 2 else 'See susceptibility data',
        'Alternative': ', '.join(infection_recommendations['Urine'][2:4]) if len(infection_recommendations['Urine']) > 2 else 'Culture-guided',
        'Duration': '3-5 days (women), 7 days (men)',
        'Notes': 'Avoid fluoroquinolones if high resistance'
    })
    
    treatment_guidelines.append({
        'Infection': 'Complicated UTI/Pyelonephritis',
        'First_Line': 'Fluoroquinolone or 3rd gen cephalosporin (if sensitive)',
        'Alternative': 'Carbapenem if MDR suspected',
        'Duration': '7-14 days',
        'Notes': 'Consider hospitalization if severe'
    })

# Wound infections
if any('Wound' in k or 'Swab' in k for k in infection_recommendations.keys()):
    wound_key = [k for k in infection_recommendations.keys() if 'Wound' in k or 'Swab' in k][0]
    treatment_guidelines.append({
        'Infection': 'Wound/Skin Infection',
        'First_Line': ', '.join(infection_recommendations[wound_key][:2]) if len(infection_recommendations[wound_key]) >= 2 else 'Beta-lactam +/- anti-MRSA',
        'Alternative': 'Culture-guided therapy',
        'Duration': '5-14 days (severity dependent)',
        'Notes': 'Add MRSA coverage if risk factors present'
    })

# General MDR
treatment_guidelines.append({
    'Infection': 'MDR Gram-negative',
    'First_Line': 'Carbapenem',
    'Alternative': 'Colistin, Tigecycline (if susceptible)',
    'Duration': 'Infection-dependent',
    'Notes': 'ID consult recommended; combination therapy may be needed'
})

treatment_guidelines.append({
    'Infection': 'MRSA',
    'First_Line': 'Vancomycin',
    'Alternative': 'Linezolid, Daptomycin',
    'Duration': 'Infection-dependent',
    'Notes': 'Monitor vancomycin trough levels'
})

guidelines_df = pd.DataFrame(treatment_guidelines)
print("\n")
print(guidelines_df.to_string(index=False))

# Export guidelines
guidelines_df.to_csv('../reports/treatment_guidelines.csv', index=False)
print("\n✓ Treatment guidelines saved to: reports/treatment_guidelines.csv")

## 8. Antibiotic Stewardship Implications

Strategic recommendations for antimicrobial stewardship program development.

In [ ]:
print("\n" + "="*70)
print("ANTIMICROBIAL STEWARDSHIP RECOMMENDATIONS")
print("="*70)

# Calculate key stewardship metrics
total_antibiotics = len(resistance_summary)
high_resistance_abs = len(resistance_summary[resistance_summary['Resistance_Rate'] >= 50])
first_line_options = len(resistance_summary[resistance_summary['Sensitivity_Rate'] >= 80])

# Load or calculate MDR data
try:
    mdr_df = pd.read_csv('../data/processed/mdr_analysis.csv')
    mdr_prevalence = (mdr_df['MDR_Status'] == 'MDR').sum() / len(mdr_df) * 100
except FileNotFoundError:
    mdr_prevalence = None

print("\n1. CURRENT STATE ASSESSMENT")
print("─"*70)
print(f"   • Total antibiotics analyzed: {total_antibiotics}")
print(f"   • High resistance antibiotics (≥50%): {high_resistance_abs} ({high_resistance_abs/total_antibiotics*100:.1f}%)")
print(f"   • First-line options available (≥80% sensitive): {first_line_options}")
if mdr_prevalence:
    print(f"   • MDR prevalence: {mdr_prevalence:.1f}%")

# Urgency assessment
print("\n2. STEWARDSHIP URGENCY LEVEL")
print("─"*70)

urgency_score = 0
if high_resistance_abs/total_antibiotics > 0.3:
    urgency_score += 2
if first_line_options < 5:
    urgency_score += 2
if mdr_prevalence and mdr_prevalence > 30:
    urgency_score += 2

if urgency_score >= 4:
    urgency = "🔴 CRITICAL - Immediate intervention required"
elif urgency_score >= 2:
    urgency = "🟡 HIGH - Urgent stewardship program needed"
else:
    urgency = "🟢 MODERATE - Standard stewardship monitoring"

print(f"   {urgency}")

# Priority interventions
print("\n3. PRIORITY INTERVENTIONS")
print("─"*70)

interventions = []

if first_line_options < 5:
    interventions.append("URGENT: Limited first-line options - implement prospective audit and feedback")

if high_resistance_abs > 5:
    interventions.append("Formulary restriction for antibiotics with >50% resistance")

if mdr_prevalence and mdr_prevalence > 25:
    interventions.append("Enhanced infection control measures - high MDR prevalence")

# Always include these
interventions.extend([
    "Implement mandatory culture collection before broad-spectrum antibiotics",
    "Develop facility-specific empiric therapy guidelines",
    "Establish automatic IV-to-PO conversion protocols",
    "Create antibiotic timeout process at 48-72 hours"
])

for i, intervention in enumerate(interventions, 1):
    print(f"   {i}. {intervention}")

# Specific targets
print("\n4. SPECIFIC STEWARDSHIP TARGETS")
print("─"*70)

# Identify problematic antibiotics
high_use_high_resistance = resistance_summary[
    (resistance_summary['Resistance_Rate'] >= 40) & 
    (resistance_summary['Total_Tests'] >= resistance_summary['Total_Tests'].median())
]

if len(high_use_high_resistance) > 0:
    print("\n   HIGH-USE ANTIBIOTICS WITH HIGH RESISTANCE:")
    print("   (Targets for restriction or pre-authorization)\n")
    for idx, row in high_use_high_resistance.head(5).iterrows():
        print(f"     • {row['Antibiotic']}: {row['Resistance_Rate']:.1f}% resistant (n={row['Total_Tests']} tests)")

# Preserve reserve agents
if len(reserve_in_data) > 0:
    print("\n   PRESERVE RESERVE ANTIBIOTICS:")
    for idx, row in reserve_in_data.iterrows():
        if row['Resistance_Rate'] > 10:
            print(f"     ⚠️  {row['Antibiotic']}: {row['Resistance_Rate']:.1f}% resistant - resistance emerging")
        else:
            print(f"     ✓ {row['Antibiotic']}: {row['Resistance_Rate']:.1f}% resistant - well preserved")

# Education targets
print("\n5. EDUCATION AND TRAINING PRIORITIES")
print("─"*70)
print("""
   • Update local antibiogram annually
   • Train clinicians on empiric therapy guidelines
   • Emphasize culture collection prior to antibiotics
   • De-escalation strategies based on culture results
   • Recognition of MDR risk factors
   • Appropriate use of reserve antibiotics
""")

# Monitoring metrics
print("\n6. KEY PERFORMANCE INDICATORS FOR MONITORING")
print("─"*70)
print("""
   PROCESS METRICS:
     □ Proportion of broad-spectrum antibiotics with documented indication
     □ Time to antibiotic de-escalation after culture results
     □ Compliance with empiric therapy guidelines
     □ Rate of culture collection before antibiotic initiation
     □ IV-to-PO conversion rate and timing
   
   OUTCOME METRICS:
     □ Changes in resistance rates (quarterly monitoring)
     □ MDR infection incidence
     □ Clinical cure rates
     □ Length of antibiotic therapy
     □ C. difficile infection rates (stewardship marker)
   
   RESOURCE METRICS:
     □ Antibiotic expenditure
     □ Days of therapy per antibiotic class
     □ Reserve antibiotic consumption
""")

## 9. Actionable Clinical Recommendations

In [ ]:
print("\n" + "="*70)
print("ACTIONABLE CLINICAL RECOMMENDATIONS")
print("="*70)

recommendations_summary = f"""
IMMEDIATE ACTIONS (Within 1 Month):
─────────────────────────────────────────────────────────────────

1. UPDATE EMPIRIC THERAPY GUIDELINES
   → Disseminate infection-specific treatment algorithms to all prescribers
   → Post antibiograms in clinical areas and EMR
   → Identify and restrict antibiotics with >50% resistance

2. IMPLEMENT CULTURE STEWARDSHIP
   → Mandate culture collection before broad-spectrum antibiotics
   → Set alerts for antibiotic orders without documented culture
   → Track culture collection rates as quality metric

3. ESTABLISH ANTIBIOTIC TIMEOUT PROTOCOL
   → Review all antibiotics at 48-72 hours
   → De-escalate based on culture results
   → Discontinue if no infection confirmed


SHORT-TERM ACTIONS (1-3 Months):
─────────────────────────────────────────────────────────────────

4. RESERVE ANTIBIOTIC PROTECTION
   → Require ID/stewardship approval for carbapenems, colistin, linezolid
   → Document indication and alternative considerations
   → Monitor reserve antibiotic consumption monthly

5. OPTIMIZE ANTIBIOTIC DELIVERY
   → Develop IV-to-PO conversion criteria
   → Implement automatic stop orders (default durations)
   → Ensure dose optimization based on renal/hepatic function

6. ENHANCE INFECTION PREVENTION
   → Reinforce hand hygiene and contact precautions for MDR organisms
   → Implement screening for MDR colonization in high-risk patients
   → Optimize catheter/device management to prevent HAIs


LONG-TERM STRATEGIC ACTIONS (3-12 Months):
─────────────────────────────────────────────────────────────────

7. PROSPECTIVE AUDIT AND FEEDBACK
   → Establish multidisciplinary stewardship team
   → Review high-risk antibiotic prescriptions within 48 hours
   → Provide prescriber-specific feedback on appropriateness

8. DIAGNOSTIC STEWARDSHIP
   → Implement rapid diagnostic tests (e.g., PCR, MALDI-TOF)
   → Use procalcitonin/biomarkers to guide therapy duration
   → Ensure timely reporting of culture results

9. CONTINUOUS SURVEILLANCE
   → Update antibiogram quarterly
   → Track resistance trends by organism and antibiotic class
   → Monitor for emerging resistance patterns
   → Report findings to prescribers and leadership

10. EDUCATION AND COMPETENCY
    → Conduct annual stewardship training for all prescribers
    → Include case-based learning on local resistance patterns
    → Certify competency in empiric therapy guideline use

"""

print(recommendations_summary)

# Specific recommendations based on data
print("\nDATA-DRIVEN SPECIFIC RECOMMENDATIONS:")
print("─"*70)

if first_line_options < 5:
    print("\n⚠️  CRITICAL: Limited first-line options detected")
    print("    → This facility has significant resistance challenges")
    print("    → Consider infectious disease consultation for complex cases")
    print("    → Implement mandatory stewardship review for all broad-spectrum antibiotics")

if len(reserve_in_data) > 0:
    avg_reserve_resistance = reserve_in_data['Resistance_Rate'].mean()
    if avg_reserve_resistance > 10:
        print("\n⚠️  WARNING: Resistance emerging to reserve antibiotics")
        print(f"    → Average resistance to reserve agents: {avg_reserve_resistance:.1f}%")
        print("    → Strict restriction policies essential to preserve these agents")
        print("    → Consider combination therapy for serious MDR infections")

# Success metrics
print("\n\nMEASURING SUCCESS:")
print("─"*70)
print("""
Target outcomes after stewardship implementation:

• 20% reduction in inappropriate broad-spectrum antibiotic use
• 90% compliance with empiric therapy guidelines
• 80% culture collection rate before antibiotics
• 50% reduction in reserve antibiotic consumption
• Stabilization or reduction in key resistance rates
• 25% reduction in antibiotic-related adverse events
• Decreased length of stay for infection-related admissions
""")

## 10. Visual Summary: Treatment Landscape

In [ ]:
# Create comprehensive treatment landscape visualization
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# 1. Sensitivity spectrum
ax1 = fig.add_subplot(gs[0, :])
sensitivity_categories = {
    'First-line\n(≥80%)': (resistance_summary['Sensitivity_Rate'] >= 80).sum(),
    'Second-line\n(60-79%)': ((resistance_summary['Sensitivity_Rate'] >= 60) & 
                               (resistance_summary['Sensitivity_Rate'] < 80)).sum(),
    'Limited utility\n(40-59%)': ((resistance_summary['Sensitivity_Rate'] >= 40) & 
                                   (resistance_summary['Sensitivity_Rate'] < 60)).sum(),
    'Avoid\n(<40%)': (resistance_summary['Sensitivity_Rate'] < 40).sum()
}

colors_spectrum = ['#27ae60', '#f39c12', '#e67e22', '#e74c3c']
bars = ax1.bar(sensitivity_categories.keys(), sensitivity_categories.values(), 
               color=colors_spectrum, alpha=0.8, edgecolor='black', linewidth=1.5)
ax1.set_ylabel('Number of Antibiotics', fontweight='bold', fontsize=11)
ax1.set_title('Antibiotic Treatment Landscape by Sensitivity Category', 
              fontweight='bold', fontsize=13)
for i, (cat, val) in enumerate(sensitivity_categories.items()):
    ax1.text(i, val + 0.5, str(val), ha='center', fontweight='bold', fontsize=12)

# 2. Top sensitive antibiotics
ax2 = fig.add_subplot(gs[1, 0])
top_sensitive = resistance_summary.nlargest(10, 'Sensitivity_Rate')
ax2.barh(range(len(top_sensitive)), top_sensitive['Sensitivity_Rate'], 
         color='#27ae60', alpha=0.7)
ax2.set_yticks(range(len(top_sensitive)))
ax2.set_yticklabels(top_sensitive['Antibiotic'], fontsize=9)
ax2.set_xlabel('Sensitivity Rate (%)', fontweight='bold')
ax2.set_title('Top 10 Most Effective Antibiotics', fontweight='bold', fontsize=11)
ax2.invert_yaxis()
ax2.set_xlim(0, 105)
for i, val in enumerate(top_sensitive['Sensitivity_Rate']):
    ax2.text(val + 1, i, f'{val:.1f}%', va='center', fontsize=8)

# 3. Top resistant antibiotics
ax3 = fig.add_subplot(gs[1, 1])
top_resistant = resistance_summary.nlargest(10, 'Resistance_Rate')
ax3.barh(range(len(top_resistant)), top_resistant['Resistance_Rate'], 
         color='#e74c3c', alpha=0.7)
ax3.set_yticks(range(len(top_resistant)))
ax3.set_yticklabels(top_resistant['Antibiotic'], fontsize=9)
ax3.set_xlabel('Resistance Rate (%)', fontweight='bold')
ax3.set_title('Top 10 Antibiotics with Highest Resistance', fontweight='bold', fontsize=11)
ax3.invert_yaxis()
ax3.set_xlim(0, 105)
for i, val in enumerate(top_resistant['Resistance_Rate']):
    ax3.text(val + 1, i, f'{val:.1f}%', va='center', fontsize=8)

# 4. Stewardship priority matrix
ax4 = fig.add_subplot(gs[2, :])
scatter_data = resistance_summary.copy()
scatter_data['Priority'] = scatter_data.apply(
    lambda x: 'High Priority Restriction' if x['Resistance_Rate'] >= 50 and x['Total_Tests'] >= scatter_data['Total_Tests'].median()
    else 'Monitor' if x['Resistance_Rate'] >= 30 
    else 'Promote Use' if x['Sensitivity_Rate'] >= 80
    else 'Standard', axis=1
)

priority_colors = {
    'High Priority Restriction': '#e74c3c',
    'Monitor': '#f39c12',
    'Promote Use': '#27ae60',
    'Standard': '#95a5a6'
}

for priority, color in priority_colors.items():
    data = scatter_data[scatter_data['Priority'] == priority]
    ax4.scatter(data['Total_Tests'], data['Resistance_Rate'], 
               c=color, label=priority, s=100, alpha=0.6, edgecolors='black')

ax4.set_xlabel('Number of Tests Performed', fontweight='bold', fontsize=11)
ax4.set_ylabel('Resistance Rate (%)', fontweight='bold', fontsize=11)
ax4.set_title('Antibiotic Stewardship Priority Matrix', fontweight='bold', fontsize=13)
ax4.axhline(50, color='red', linestyle='--', alpha=0.5, label='High resistance threshold')
ax4.legend(loc='upper right', framealpha=0.9)
ax4.grid(True, alpha=0.3)

plt.suptitle('Antimicrobial Treatment Options - Comprehensive Summary', 
             fontsize=16, fontweight='bold', y=0.995)

plt.savefig('../reports/figures/treatment_landscape_summary.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Comprehensive treatment landscape visualization saved")

## Summary and Conclusions

### Key Findings:

**1. Available Treatment Options**
   - First-line antibiotics with ≥80% sensitivity identified for empiric therapy
   - Reserve antibiotics characterized for MDR cases
   - Infection-specific recommendations developed based on local susceptibility patterns

**2. Clinical Decision Support**
   - Evidence-based treatment algorithms created for common infections
   - Organism-specific antibiotic recommendations provided
   - MDR risk stratification incorporated into treatment selection

**3. Antimicrobial Stewardship Priorities**
   - High-resistance antibiotics identified for restriction or pre-authorization
   - Culture stewardship emphasized to enable targeted therapy
   - De-escalation strategies highlighted to preserve broad-spectrum agents
   - Reserve antibiotic preservation critical to maintain treatment options

**4. Quality Improvement Targets**
   - Empiric therapy guideline compliance
   - Culture collection prior to antibiotic initiation
   - Timely de-escalation based on susceptibility results
   - Appropriate use of reserve antibiotics
   - Reduction in antibiotic-related adverse events

### Clinical Impact:

Implementation of these evidence-based treatment recommendations will:
- **Optimize patient outcomes** through appropriate empiric therapy selection
- **Reduce treatment failures** by avoiding antibiotics with high local resistance
- **Preserve critical antimicrobials** through judicious use of reserve agents
- **Support antimicrobial stewardship** with data-driven interventions
- **Improve antibiotic prescribing** through decision support tools

### Next Steps:

1. **Immediate Implementation**
   - Disseminate infection-specific treatment guidelines
   - Update institutional antibiogram
   - Establish antibiotic timeout protocols

2. **Stewardship Program Development**
   - Form multidisciplinary stewardship team
   - Implement prospective audit and feedback
   - Establish metrics for continuous monitoring

3. **Ongoing Surveillance**
   - Quarterly resistance rate monitoring
   - Annual antibiogram updates
   - Track stewardship intervention outcomes

---

**Clinical Perspective:** This analysis transforms local susceptibility data into actionable treatment recommendations. By aligning empiric therapy with resistance patterns and implementing stewardship principles, we can optimize patient outcomes while preserving our antimicrobial armamentarium for future generations.

**Data-Driven Care:** Every antibiotic choice should be informed by local resistance patterns. This notebook provides the evidence base for rational antibiotic selection in our patient population.